In [ ]:
import numpy as np
import cv2
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os,cvxopt
from cvxopt import matrix, solvers

In [ ]:
data = []
labels = []

classes = ['1','2']

dataset_path = 'svm/train/'
for x in classes:
    path = os.path.join(dataset_path, x)
    for y in os.listdir(path):
        image = cv2.imread(os.path.join(path, y))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (16,16))
        image = image.reshape(768)
        image=image/255.0
        data.append(image)
        labels.append(float(2*int(x)-3))
x_train = np.array(data)
y_train = np.array(labels)

In [ ]:
test_data = []
test_labels=[]

classes = ['1','2']

dataset_path = 'svm/val/'
for x in classes:
    path = os.path.join(dataset_path, x)
    for y in os.listdir(path):
        image = cv2.imread(os.path.join(path, y))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (16,16))
        image = image.reshape(768)
        image=image/255.0
        test_data.append(image)
        test_labels.append(float(2*int(x)-3))
x_test = np.array(test_data)
y_test = np.array(test_labels)

In [ ]:
def gaussian(X, gamma):
    squared_distances = np.sum(X**2, axis=1).reshape(-1, 1) + np.sum(X**2, axis=1) - 2 * np.dot(X, X.T)
    K = np.exp(-gamma * squared_distances)
    return K
def Kernel(X,gamma):
    if gamma:
        return gaussian(X, gamma)
    else:
        return np.dot(X, X.T)

def find_P(X, Y, gamma):
    K = Kernel(X, gamma)
    P=np.outer(Y,Y)*K
    return P
    

In [ ]:
m = x_train.shape[0]
P = find_P(x_train,y_train, 0.001)
P = matrix(P)

q = -np.ones((m, 1))
q = matrix(q)

P,q

In [ ]:

A = matrix(y_train.reshape(1, -1), tc='d')

B = matrix(0.0, tc='d')

A,B

In [ ]:
# h1=np.zeros((data.shape[0],1))

G = np.vstack((-np.eye(m), np.eye(m)))
G = matrix(G)

C=1.0
h = np.zeros((2 * m, 1))
h[m:, :] = C   
h = matrix(h)

G,h

In [ ]:

sol = solvers.qp(P, q,G, h, A, B,options={'show_progress': False}) 

In [ ]:
print(len(sol['x']))
sol['primal objective']

In [ ]:
alpha = np.reshape(np.array(sol['x']), (x_train.shape[0],1))
indices = [i for i in range(alpha.shape[0]) if alpha[i] > 1e-5]
# alpha = alpha[indices]
len(alpha)

In [ ]:
def find_b(alpha,X,Y):
    n = len(X)
    b=np.sum(Y)
    for i in range(n):
        b+=np.sum(alpha*Y*np.exp(-0.001*(np.linalg.norm(X[i]-X)**2)))
    return b/len(alpha)
b  = find_b(alpha,x_train[indices],y_train[indices])
b

In [ ]:
def no_of_sv(alpha,C):
    SVs = np.where((alpha > 1e-5) & (alpha < C))[0]
    return len(SVs), SVs
# no_of_sv(alpha,1.0)

In [ ]:
# alpha.shape, data.shape, labels.shape
svi = np.where((alpha > 1e-5) & (alpha < C ))[0]
W = sum(alpha[i]*labels[i]*data[i] for i in svi)
W = np.array(W).reshape((x_train.shape[1],1))
W.shape

In [ ]:
import matplotlib.pyplot as plt
svi = np.where((alpha > 1e-5) & (alpha < C))[0]
top_svi = svi[np.argsort(np.abs(alpha[svi]))[-6:][::-1]]

plt.figure(figsize=(10, 6))

for i, idx in enumerate(top_svi):
    support_vector = data[idx]

    support_vector_image = support_vector.reshape((16, 16, 3))

    plt.subplot(2, 3, i + 1)  
    plt.imshow(support_vector_image)
    plt.title(f"Support Vector {i+1}")

plt.tight_layout()
plt.show()


In [ ]:
b = 0.0
svi = np.where((alpha > 1e-5) & (alpha < C ))[0]
for idx in svi:
    b += labels[idx] - np.dot(W.T, data[idx])

b /= len(svi)
b

In [ ]:
test_data = []
test_labels=[]

classes = ['1','2']

dataset_path = 'svm/val/'
for x in classes:
    path = os.path.join(dataset_path, x)
    for y in os.listdir(path):
        image = cv2.imread(os.path.join(path, y))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (16,16))
        image = image.reshape(768)
        image=image/255.0
        test_data.append(image)
        test_labels.append(float(2*int(x)-3))
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [ ]:
def predict(W,b,x_test,y_test):
    y_pred=[]
    for i in range(len(x_test)):
        y_pred.append(np.sign(np.dot(W.T,x_test[i])+b))
    # y_pred = [(y+3)//2 for y in y_pred]
    y_pred=np.array(y_pred)
    y_pred=y_pred.reshape(y_test.shape)
    print("accuracy: ",accuracy_score(y_test,y_pred))
    return y_pred
predict(W,b,x_train,y_train)

In [ ]:
predict(W,b,x_test,y_test)

In [ ]:
# import libsvm
# from libsvm import svmutil
from libsvm.svmutil import svm_problem,svm_parameter,svm_train,svm_predict


In [ ]:
def sv_libsvm(x_train,y_train,kernel,gamma):
    prob = svm_problem(y_train,x_train)
    param = svm_parameter('-t {} -c 1'.format(kernel))
    if gamma:
        param = svm_parameter('-t {} -c 1 -g {}'.format(kernel,gamma))
    model = svm_train(prob,param)
    return model
def accuracy_libsvm(model,x_train,y_train,x_test,y_test):
    p_label,p_acc_train,p_val = svm_predict(y_train,x_train,model)
    print("Training Accuracy: ",p_acc_train[0])
    p_label,p_acc_validation,p_val = svm_predict(y_test,x_test,model)
    print("Validation Accuracy: ",p_acc_validation[0])
    return p_acc_train,p_acc_validation[0]

In [ ]:
indices, alpha, model, b = sv_libsvm(x_train,x_test,y_train,y_test,0.0)

In [ ]:
accuracy_libsvm(data,list(labels),test_data,list(test_labels),model)

In [ ]:
w = sum(alpha[i]*labels[i]*data[i] for i in range(len(alpha)))
w = np.array(w).reshape((data.shape[1],1))

w

In [ ]:
data_ex = []
labels_ex = []

classes = ['1','2']

dataset_path = 'svm/train/'
for x in classes:
    path = os.path.join(dataset_path, x)
    for y in os.listdir(path):
        image = cv2.imread(os.path.join(path, y))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (32,32))
        image = image.reshape(3072)
        image=image/255.0
        data_ex.append(image)
        labels_ex.append(float(2*int(x)-3))
data_ex = np.array(data_ex)
labels_ex = np.array(labels_ex)

In [ ]:
test_data_ex = []
test_labels_ex = []

classes = ['1','2']

dataset_path = 'svm/train/'
for x in classes:
    path = os.path.join(dataset_path, x)
    for y in os.listdir(path):
        image = cv2.imread(os.path.join(path, y))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (32,32))
        image = image.reshape(3072)
        image=image/255.0
        test_data_ex.append(image)
        test_labels_ex.append(float(2*int(x)-3))
test_data_ex = np.array(test_data_ex)
test_labels_ex = np.array(test_labels_ex)

In [ ]:
indices, alpha, model, b = sv_libsvm(data_ex,test_data_ex,labels_ex,test_labels_ex,0.001)

In [ ]:
from collections import Counter
Counter(svm_predict(y_train, x_train, model, '-q')[0])

In [ ]:
accuracy_libsvm(data_ex,list(labels_ex),test_data_ex,list(test_labels_ex),model)

In [ ]:
kC2classes = [['1','2'],['1','3'],['1','4'],['1','5'],['1','6'],
              ['2','3'],['2','4'],['2','5'],['2','6'],['3','4'],
              ['3','5'],['3','6'],['4','5'],['4','6'],['5','6']]


In [ ]:
for x in kC2classes:
    data = []
    labels = []

    classes = x

    dataset_path = 'svm/train/'
    for x in classes:
        path = os.path.join(dataset_path, x)
        for y in os.listdir(path):
            image = cv2.imread(os.path.join(path, y))
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (16,16))
            image = image.reshape(768)
            image=image/255.0
            data.append(image)
            labels.append(float(2*int(x)-3))
    data = np.array(data)
    labels = np.array(labels)
    m = data.shape[0]
    P = find_P(data,labels, 0)
    P = matrix(P)

    q = -np.ones((m, 1))
    q = matrix(q)

In [ ]:
def kernel(x_i,x_j):
    return np.exp(-0.001*(np.linalg.norm(x_i-x_j)**2))
def wtx(alpha,Y,X,x_j,indices):
    return sum(alpha[i]*Y[i]*kernel(X[i],x_j) for i in indices)


In [ ]:
b = 0.0
svi = np.where((alpha > 1e-5) & (alpha < C ))[0]
for i in svi:
    b += labels[i] - wtx(alpha,y_train,x_train,x_train[i],svi)
b/=len(svi)

In [ ]:
b

In [ ]:
def predict(alpha,x_train,y_train,b,x_test,y_test):
    y_pred=[]
    for i in range(len(x_test)):
        y_pred.append((wtx(alpha,y_train,x_train,x_test[i],indices)+b))
    # print(y_pred)
    return np.sign(y_pred)

In [ ]:
y_pred= predict(alpha,x_train,y_train,b,x_test,y_test)
print(len(y_pred))

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
def gauss_kernel(X1, X2, gamma):
    prod1 = np.reshape(np.einsum('ij,ij->i', X1, X1), (X1.shape[0], 1))
    prod2 = np.reshape(np.einsum('ij,ij->i', X2, X2), (X2.shape[0], 1))
    prod = prod1 + prod2.T - 2 * np.matmul(X1, X2.T)
    return np.exp(-gamma * prod)
